Importing the fork reddgr/pinecone-datasets with fixed issue when running on Windows: https://github.com/pinecone-io/pinecone-datasets/issues/35

In [ ]:
!git clone https://github.com/reddgr/pinecone-datasets

import os
import platform
print(platform.system())
if platform.system() == "Windows":
    os.system("rename pinecone-datasets pinecone_datasets_reddgr")
    os.system("ren pinecone_datasets_reddgr\\pinecone_datasets pinecone_datasets_reddgr")
    os.system("rmdir /s /q pinecone_datasets_reddgr\\.github")
    os.system("del pinecone_datasets_reddgr\\.git")
    os.system("del pinecone_datasets_reddgr\\.gitignore")

Windows


Cloning into 'pinecone-datasets'...


[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/pinecone-io/examples/blob/master/docs/semantic-search.ipynb) [![Open nbviewer](https://raw.githubusercontent.com/pinecone-io/examples/master/assets/nbviewer-shield.svg)](https://nbviewer.org/github/pinecone-io/examples/blob/master/docs/semantic-search.ipynb)

# Semantic Search

In [3]:
import os
from dotenv import load_dotenv
from pinecone import Pinecone
load_dotenv('C:/apis/.env') # path to your dotenv file
api_key = os.getenv("PINECONE_API_KEY")
def mask_token(token, unmasked_chars=4):
    return token[:unmasked_chars] + '*' * (len(token) - unmasked_chars*2) + token[-unmasked_chars:]
print(mask_token(api_key,4))

# configure client
pc = Pinecone(api_key=api_key)

pcsk*******************************************************************FZVA


c:\Users\david\Documents\git\SemanticSearch\venv\Lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


## Data Download

In this notebook we will skip the data preparation steps as they can be very time consuming and jump straight into it with the prebuilt dataset from *Pinecone Datasets*. If you'd rather see how it's all done, please refer to [this notebook](https://github.com/pinecone-io/examples/blob/master/learn/search/semantic-search/semantic-search.ipynb).

Let's go ahead and download the dataset.

In [4]:
# from pinecone_datasets import load_dataset
import sys
sys.path.append('pinecone_datasets_reddgr')
from pinecone_datasets_reddgr import load_dataset
dataset = load_dataset('quora_all-MiniLM-L6-bm25')

dataset = load_dataset('quora_all-MiniLM-L6-bm25')
# we drop metadata as will use blob column
dataset.documents.drop(['metadata'], axis=1, inplace=True)
dataset.documents.rename(columns={'blob': 'metadata'}, inplace=True)
# we will use 80K rows of the dataset between rows 240K -> 320K
dataset.documents.drop(dataset.documents.index[320_000:], inplace=True)
dataset.documents.drop(dataset.documents.index[:240_000], inplace=True)
dataset.head()
print(len(dataset))

read_path_str: gs://pinecone-datasets-dev/quora_all-MiniLM-L6-bm25/documents/*.parquet
data_type_path: gs://pinecone-datasets-dev/quora_all-MiniLM-L6-bm25/documents
80000


In [5]:
dataset_df = dataset.documents
dataset_df.sample(5)

,id,values,sparse_values,metadata
298963,92660,"[0.092688106, 0.001267088, -0.0035857065, -0.0...","{'indices': [3844, 3865, 8564, 17227, 18688, 2...",{'text': ' Why do parents not want their teena...
297219,90864,"[-0.028288627, -0.0038225593, 0.024150297, 0.0...","{'indices': [5161, 20213, 23463, 27512, 29387,...",{'text': ' How does Ichigo controls his full h...
262695,55238,"[0.05329664, 0.069775075, -0.0060466942, -0.04...","{'indices': [6296, 13677, 23041, 29100, 32936,...",{'text': ' What is wrong with banning Pakistan...
280777,73875,"[-0.10754486, -0.013177942, -0.037665408, 0.02...","{'indices': [7582, 9198, 22014, 31925, 36821, ...",{'text': ' How do I unblock someone on Instagr...
251584,527839,"[0.047862735, -0.0094146, 0.06145957, 0.048052...","{'indices': [8564, 23463, 33337, 35433, 39832,...",{'text': ' Why does Columbia not keep NoCo ope...


## Creating an Index

Now the data is ready, we can set up our index to store it.

We begin by initializing our connection to Pinecone. To do this we need a [free API key](https://app.pinecone.io).

In [9]:
import os
from dotenv import load_dotenv
from pinecone import Pinecone
load_dotenv('C:/apis/.env') # path to your dotenv file
api_key = os.getenv("PINECONE_API_KEY")
def mask_token(token, unmasked_chars=4):
    return token[:unmasked_chars] + '*' * (len(token) - unmasked_chars*2) + token[-unmasked_chars:]
print(mask_token(api_key,4))

# configure client
pc = Pinecone(api_key=api_key)

existing_indexes = [
    index_info["name"] for index_info in pc.list_indexes()
]
print("Existing indexes:\n",existing_indexes)

pcsk*******************************************************************FZVA
Existing indexes:
 ['semantic-search-fast']


In [8]:
existing_indexes = [
    index_info["name"] for index_info in pc.list_indexes()
]
print(existing_indexes)

['semantic-search-fast']


Now we setup our index specification, this allows us to define the cloud provider and region where we want to deploy our index. You can find a list of all [available providers and regions here](https://docs.pinecone.io/docs/projects).

In [17]:
from pinecone import ServerlessSpec

cloud = os.environ.get('PINECONE_CLOUD') or 'aws'
region = os.environ.get('PINECONE_REGION') or 'us-east-1'

spec = ServerlessSpec(cloud=cloud, region=region)

Now we create a new index called `semantic-search-fast`. It's important that we align the index `dimension` and `metric` parameters with those required by the `MiniLM-L6` model.

In [18]:
index_name = 'semantic-search-fast'

In [19]:
import time

existing_indexes = [
    index_info["name"] for index_info in pc.list_indexes()
]

# check if index already exists (it shouldn't if this is first time)
if index_name not in existing_indexes:
    # if does not exist, create index
    pc.create_index(
        index_name,
        dimension=384,  # dimensionality of minilm
        metric='dotproduct',
        spec=spec
    )
    # wait for index to be initialized
    while not pc.describe_index(index_name).status['ready']:
        time.sleep(1)

# connect to index
index = pc.Index(index_name)
time.sleep(1)
# view index stats
index.describe_index_stats()

{'dimension': 384,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 80000}},
 'total_vector_count': 80000}

Upsert the data:

In [20]:
from tqdm.auto import tqdm

for batch in tqdm(dataset.iter_documents(batch_size=500), total=160):
    index.upsert(batch)

100%|██████████| 160/160 [04:59<00:00,  1.87s/it]


## Load an index (if it already exists)

In [10]:
index_name = 'semantic-search-fast'
index = pc.Index(index_name)

## Making Queries

Now that our index is populated we can begin making queries. We are performing a semantic search for *similar questions*, so we should embed and search with another question. Let's begin.

In [11]:
from sentence_transformers import SentenceTransformer
import torch

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Using device:', device)

model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2', device=device)
model

Using device: cpu


SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
)

Now let's query.

In [12]:
query = "what is love?" # this is a question

# create the query vector
xq = model.encode(query).tolist()

# now query
xc = index.query(vector=xq, top_k=5, include_metadata=True)
xc

{'matches': [{'id': '520867',
              'metadata': {'text': ' What is love?'},
              'score': 0.999236286,
              'values': []},
             {'id': '95783',
              'metadata': {'text': ' What is love?'},
              'score': 0.999236,
              'values': []},
             {'id': '57153',
              'metadata': {'text': ' What is meaning of love?'},
              'score': 0.882388592,
              'values': []},
             {'id': '83493',
              'metadata': {'text': ' What does is it mean to love?'},
              'score': 0.820300698,
              'values': []},
             {'id': '57154',
              'metadata': {'text': ' What is the exact meaning of love?'},
              'score': 0.818767905,
              'values': []}],
 'namespace': '',
 'usage': {'read_units': 7}}

In the returned response `xc` we can see the most relevant questions to our particular query — we don't have any exact matches but we can see that the returned questions are similar in the topics they are asking about. We can reformat this response to be a little easier to read:

In [13]:
for result in xc['matches']:
    print(f"{round(result['score'], 2)}: {result['metadata']['text']}")

1.0:  What is love?
1.0:  What is love?
0.88:  What is meaning of love?
0.82:  What does is it mean to love?
0.82:  What is the exact meaning of love?


These are good results, let's try and modify the words being used to see if we still surface similar results.

In [14]:
query = "I wanna know what love is" # this is a request

# create the query vector
xq = model.encode(query).tolist()

# now query
xc = index.query(vector=xq, top_k=5, include_metadata=True)
for result in xc['matches']:
    print(f"{round(result['score'], 2)}: {result['metadata']['text']}")

0.81:  What is love?
0.81:  What is love?
0.79:  What is love? How can we find that we are in love?
0.74:  What is meaning of love?
0.73:  What is the exact meaning of love?


## Delete index

In [ ]:
pc.delete_index(index_name)

---